In [1]:
import os
os.chdir("../")

In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/udit-rawat/CardioX.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "udit-rawat"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "8acd2150d81505b961f1bd33e87011359565e760"

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [4]:
from src.mlpro.constants import *
from src.mlpro.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.xgboost_param
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/udit-rawat/CardioX.mlflow",

        )

        return model_evaluation_config

In [6]:
import os
import pandas as pd
from sklearn.metrics import precision_score, f1_score, recall_score, roc_auc_score, matthews_corrcoef
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from pathlib import Path
import json

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_classification_metrics(self, actual, pred, pred_proba):
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        roc_auc = roc_auc_score(actual, pred_proba)
        mcc = matthews_corrcoef(actual, pred)
        return precision, recall, f1, roc_auc, mcc



    def log_into_mlflow(self):
        # Load the model and data
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column], axis=1).to_numpy()
        test_y = test_data[self.config.target_column].values

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            # Make predictions using XGBoost
            y_pred_proba = model.predict_proba(test_x)[:, 1]  # Probabilities
            y_pred = (y_pred_proba >= 0.5).astype(
                int)        # Binary predictions

            # Evaluate classification metrics
            precision, recall, f1, roc_auc, mcc = self.eval_classification_metrics(
                test_y, y_pred, y_pred_proba)

            # Save metrics as local
            scores = {
                "precision": precision, "recall": recall, "f1_score": f1,
                "roc_auc": roc_auc, "mcc": mcc
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log all parameters and classification metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("roc_auc", roc_auc)
            mlflow.log_metric("mcc", mcc)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    model, "model", registered_model_name="XGBClassifier"
                )
            else:
                mlflow.sklearn.log_model(model, "model")

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-10-13 23:42:04,757: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-13 23:42:04,759: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-13 23:42:04,762: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-13 23:42:04,764: INFO: common: created directory at: artifacts]
[2024-10-13 23:42:04,765: INFO: common: created directory at: artifacts/model_evaluation]
[2024-10-13 23:42:05,510: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/Users/uditrawat/Desktop/CardioX/cardiox-e/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'XGBClassifier'.
2024/10/13 23:42:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBClassifier, version 1
Created version '1' of model 'XGBClassifier'.
